In [1]:
import tensorflow as tf
from tensorflow.python.keras.datasets import cifar10
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import layers, activations, optimizers, models,losses
import os
import matplotlib.pyplot as plt

tf.logging.set_verbosity(tf.logging.INFO)

c:\programdata\anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Model Parameters

In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), "saved_models")
model_name = 'keras_cifar10_trained_model.h5'

# Load data into memory

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Define Model

In [7]:
def model_fn(features, labels, mode):
    inputs = tf.convert_to_tensor(features["x"])

    #Convolution Layer-1 
    conv1 = tf.layers.conv2d(
        inputs=inputs,
        filters=32,
        kernel_size=[3, 3],
        activation=tf.nn.relu,
        padding="same",
        name="Conv-1.1"
    )
    conv1 = tf.layers.conv2d(
        inputs=conv1,
        filters=32,
        kernel_size=[3, 3],
        activation=tf.nn.relu,
        padding="same",
        name="Conv-1.2"
    )
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], padding="same", strides=1,
                                    name="Max_Pool-1")
    dr1 = tf.layers.dropout(pool1, rate=0.25, name="drop-1")

    #Convolution Layer-2 
    conv2 = tf.layers.conv2d(
        inputs=dr1,
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu,
        padding="same",
        name="Conv-2.1"
    )
    conv2 = tf.layers.conv2d(
        inputs=conv2,
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu,
        padding="same",
        name="Conv-2.2"
    )
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], padding="same", strides=1,
                                    name="Max_Pool-2")
    dr2 = tf.layers.dropout(pool2, rate=0.25, name="drop-2")

    pool2flat = tf.layers.flatten(dr2, name="flattern")
    dense = tf.layers.dense(inputs=pool2flat, units=512, activation=tf.nn.relu)
    dr_dense = tf.layers.dropout(dense, rate=0.25, name="drop-3")

    logits = tf.layers.dense(inputs=dr_dense, units=num_classes)

    predictions = {
        "classes": tf.argmax(logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="Softmax_Tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=labels)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, eval_metric_ops=eval_metric, loss=loss)


Define Input Pipeline


In [9]:
eval_ipfn = tf.estimator.inputs.numpy_input_fn(
    x= {"x":x_test},
    y = y_test,
    shuffle=False
)

train_ipfn = tf.estimator.inputs.numpy_input_fn(
    x= {"x":x_train},
    y = y_train,
    batch_size=32,
    num_epochs=epochs,
    shuffle=True
)
cifar_model = tf.estimator.Estimator(model_fn=model_fn,model_dir="./models")

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': './models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FF08FF0F28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
my_checkpoint_config = tf.estimator.RunConfig(save_checkpoints_steps=100)
tensors_to_log = {"probabilities": "Softmax_Tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

# cifar_model.train(train_ipfn, steps=2, hooks=[logging_hook])

cifar_model.train(train_ipfn, steps=2)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./models\model.ckpt-2


...

INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 3 into ./models\model.ckpt.


INFO:tensorflow:loss = 2.3101068, step = 3


INFO:tensorflow:Saving checkpoints for 4 into ./models\model.ckpt.


INFO:tensorflow:Loss for final step: 2.3050284.


In [12]:
eval_result = cifar_model.evaluate(input_fn=eval_ipfn)
print(eval_result)

INFO:tensorflow:Calling model_fn.


ValueError: Can not squeeze dim[1], expected a dimension of 1, got 10 for 'remove_squeezable_dimensions/Squeeze' (op: 'Squeeze') with input shapes: [?,10].